
# Bayesian Optimization with Retrieval Optimizer

This notebook demonstrates how to use Bayesian optimization to tune Redis-based retrieval pipelines. Unlike a grid study—which tests all combinations—Bayesian optimization intelligently searches the configuration space, prioritizing promising settings based on previous results. This is especially useful when the number of possible configurations is large and exhaustive search would be too costly.

You'll define a study configuration, specify embedding models and search methods, and let the optimizer guide the search toward the best-performing retrieval setup.


## Dataset

We'll import a dataset from the [beir benchmark IR project](https://github.com/beir-cellar/beir) to get going quickly. 



In [ ]:
# Load data
from redis_retrieval_optimizer.corpus_processors import eval_beir

# check the link above for different datasets to try
beir_dataset_name = "nfcorpus"

# Load sample data
corpus, queries, qrels = eval_beir.get_beir_dataset(beir_dataset_name)

## Study config

In this directory there is a yaml file containing a configuration for a bayesian study that looks like this:

```yaml
# path to data files for easy read
corpus: "data/nfcorpus_corpus.json"
queries: "data/nfcorpus_queries.json"
qrels: "data/nfcorpus_qrels.json"

index_settings:
  name: "optimize"
  vector_field_name: "vector" # name of the vector field to search on
  text_field_name: "text" # name of the text field for lexical search
  from_existing: false
  vector_dim: 384 # should match first embedding model or from_existing
  additional_fields:
      - name: "title"
        type: "text"

optimization_settings:
  # defines the options optimization can take
  metric_weights:
    f1_at_k: 1
    embedding_latency: 1
    total_indexing_time: 1
  algorithms: ["hnsw"]
  vector_data_types: ["float16", "float32"]
  distance_metrics: ["cosine"]
  n_trials: 10
  n_jobs: 1
  ret_k: [1, 10] # potential range of value to be sampled during study
  ef_runtime: [10, 20, 30, 50]
  ef_construction: [100, 150, 200, 250, 300]
  m: [8, 16, 64]


search_methods: ["vector", "lin_combo"]
embedding_models:
  - type: "hf"
    model: "sentence-transformers/all-MiniLM-L6-v2"
    dim: 384
    embedding_cache_name: "vec-cache" # avoid names with including 'ret-opt' as this can cause collisions
    dtype: "float32"
```

## Running a study

To run a study simple pass the path to config, redis_url, and corpus processing function to the `run_bayes_study` function and the package will take care of the rest. 

In [1]:
# run study

# add root redis_retrieval_optimizer to path until available on pypi
import sys
import os

# Get the current notebook directory
current_dir = os.path.dirname(os.path.abspath(''))

# Go up two directory levels (adjust the number as needed)
parent_dir = os.path.abspath(os.path.join(current_dir, '../..'))

# Add the parent directory to the Python path if it's not already there
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)
    print(f"Added {parent_dir} to Python path")


import os
from redis_retrieval_optimizer.bayes_study import run_bayes_study
from redis_retrieval_optimizer.corpus_processors import eval_beir
from dotenv import load_dotenv

# load environment variables containing necessary credentials
load_dotenv()

redis_url = os.environ.get("REDIS_URL", "redis://localhost:6379/0")

metrics = run_bayes_study(
    config_path="bayes_study_config.yaml",
    redis_url=redis_url,
    corpus_processor=eval_beir.process_corpus
)

/Users/robert.shelton/Library/Caches/pypoetry/virtualenvs/redis-retrieval-optimizer-OYoDaORV-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


10:24:57 sentence_transformers.cross_encoder.CrossEncoder INFO   Use pytorch device: mps


[I 2025-05-13 10:24:58,125] A new study created in memory with name: test


10:24:58 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: mps
10:24:58 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Batches: 100%|██████████| 1/1 [00:00<00:00,  8.27it/s]


...

10:26:22 redisvl.index.index INFO   Index already exists, overwriting.
10:26:22 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: mps
10:26:22 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Batches: 100%|██████████| 1/1 [00:00<00:00, 67.50it/s]


Skip recreate
10:26:23 root INFO   Indexing progress: 0.46030252587392345
10:26:25 root INFO   Indexing progress: 0.6520228703770717
10:26:26 root INFO   Indexing progress: 0.8399797351089238
10:26:27 root INFO   Indexing progress: 1
10:26:27 root INFO   Data indexed total_indexing_time=4.745s, num_docs=6266


 WARNING num_docs=6266 != corpus_size=1144... Settings recreate_index=True, recreate_data=False
10:26:28 root INFO   Saving metrics for study: bc651995-539d-404e-9d83-67759f1f1c81, METRICS={'search_method': ['lin_combo', 'lin_combo', 'lin_combo', 'lin_combo', 'vector', 'lin_combo', 'vector', 'vector', 'lin_combo', 'vector'], 'total_indexing_time': [3.301, 3.319, 4.995, 7.718, 7.542, 5.977, 4.816, 3.903, 3.948, 4.745], 'avg_query_time': [0.004180300715538001, 0.004311325380307602, 0.0042008593355539045, 0.004657205782438579, 0.001498519820694584, 0.0041912170386535835, 0.001270003363074902, 0.0009956765839189937, 0.0015962736525402719, 0.0015867472433084306], 'model': ['sentence-t

[I 2025-05-13 10:26:28,065] Trial 9 finished with value: 4.745 and parameters: {'model_info': {'type': 'hf', 'model': 'sentence-transformers/all-MiniLM-L6-v2', 'dim': 384, 'embedding_cache_name': 'vec-cache', 'dtype': 'float32'}, 'search_method': 'vector', 'algorithm': 'hnsw', 'var_dtype': 'float32', 'distance_metric': 'cosine', 'ret_k': 8, 'ef_runtime': 50, 'ef_construction': 150, 'm': 16}. Best is trial 3 with value: 7.718.


Completed Bayesian optimization... 


Best Configuration: 3: {'model_info': {'type': 'hf', 'model': 'sentence-transformers/all-MiniLM-L6-v2', 'dim': 384, 'embedding_cache_name': 'vec-cache', 'dtype': 'float32'}, 'search_method': 'lin_combo', 'algorithm': 'hnsw', 'var_dtype': 'float32', 'distance_metric': 'cosine', 'ret_k': 4, 'ef_runtime': 50, 'ef_construction': 150, 'm': 16}:


Best Score: [7.718]




In [2]:
metrics[["search_method", "algorithm", "vector_data_type", "ef_construction", "ef_runtime", "m", "avg_query_time", "recall@k", "precision", "ndcg@k"]].sort_values(by="ndcg@k", ascending=False)

,search_method,algorithm,vector_data_type,ef_construction,ef_runtime,m,avg_query_time,recall@k,precision,ndcg@k
0,lin_combo,hnsw,float32,300,30,64,0.004180,0.154988,0.243344,0.203651
1,lin_combo,hnsw,float16,150,50,8,0.004311,0.154988,0.243344,0.203651
2,lin_combo,hnsw,float16,200,30,64,0.004201,0.154988,0.243344,0.203647
3,lin_combo,hnsw,float32,150,50,16,0.004657,0.128272,0.283801,0.182487
5,lin_combo,hnsw,float32,200,10,64,0.004191,0.128272,0.283801,0.181987
4,vector,hnsw,float32,300,50,64,0.001499,0.126747,0.282316,0.169387
9,vector,hnsw,float32,150,50,16,0.001587,0.126360,0.281948,0.168807
7,vector,hnsw,float32,100,30,16,0.000996,0.120460,0.278383,0.163599
6,vector,hnsw,float32,250,20,8,0.001270,0.106727,0.265248,0.147901
8,lin_combo,hnsw,float32,150,20,8,0.001596,0.099280,0.262494,0.146263


In [9]:
metrics.columns

Index(['search_method', 'total_indexing_time', 'avg_query_time', 'model',
       'model_dim', 'ret_k', 'recall@k', 'ndcg@k', 'f1@k', 'precision',
       'algorithm', 'ef_construction', 'ef_runtime', 'm', 'distance_metric',
       'vector_data_type'],
      dtype='object')